# Compiling Circuits for EeroQ via Cirq

## Import Requirements

This tutorial will showcase how to compile a circuit for EeroQ hardware using the ```cirq-superstaq``` client. 

In [1]:
# Required imports
try:
    import cirq
    import cirq_superstaq as css
except ImportError:
    print("Installing cirq-superstaq...")
    %pip install --quiet 'cirq-superstaq[examples]'
    print("Installed cirq-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import cirq
    import cirq_superstaq as css

# Optional imports
import numpy as np
import os  # Used if setting a token as an environment variable

%load_ext autoreload
%autoreload 2

To interface Superstaq via Cirq, we must first instantiate a service provider in ```cirq-superstaq``` with ```Service()```. We then supply a Superstaq API key (which you can get from https://superstaq.infleqtion.com) by either providing the API key as an argument of Service, i.e., ```css.Service(api_key="token")```, or by setting it as an environment variable. (see more details [here](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_css.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [2]:
service = css.Service()

# EeroQ Gates

In [3]:
dd_gate = css.DDPowGate(exponent=1)
cirq.Circuit(dd_gate.on(cirq.q(0), cirq.q(1)))

0: ───DD───
      │
1: ───DD───

In [4]:
cirq.unitary(dd_gate)

array([[ 0.-1.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j]])

Below is the EeroQ protocol for a CZ Gate.

![title](images/DD_gate.png)

In [5]:
circuit = cirq.Circuit(cirq.CZ(cirq.q(0), cirq.q(1)))
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│──────────────────────────────
           │
0-: ───X───│───DD────────S──────DD────────
           │   │                │
1+: ───────│───┼─────────Z──────┼─────────
           │   │                │
1-: ───X───│───DD^0.25───S^-1───DD^0.25───

In [6]:
def compute_unitary(circuit: cirq.Circuit):
    """Computes the n*n unitary of a 2n electron EeroQ circuit"""
    unitary = cirq.unitary(circuit[1:]).reshape((4,) * cirq.num_qubits(circuit))
    mat = unitary[tuple(slice(1, 3) for _ in range(cirq.num_qubits(circuit)))]
    dim = round(np.sqrt(mat.size))
    mat = mat.reshape(dim, dim)
    return mat

In [7]:
mat = compute_unitary(compiled_circuit)
mat/mat[0][0]

array([[ 1.-0.00000000e+00j, -0.+0.00000000e+00j, -0.+0.00000000e+00j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j,  1.+7.85046229e-17j, -0.+0.00000000e+00j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j, -0.+0.00000000e+00j,  1.+7.85046229e-17j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j, -0.+0.00000000e+00j, -0.+0.00000000e+00j,
        -1.-0.00000000e+00j]])

In [8]:
cirq.allclose_up_to_global_phase(cirq.unitary(circuit), mat)

True

In [9]:
circuit += cirq.measure(cirq.q(0), cirq.q(1))
job = service.create_job(circuit, target= "eeroq_wonderlake_qpu", repetitions = 10, method="dry-run")

In [10]:
job.counts()

[{'00': 10}]

# Circuit Compilation

In [11]:
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(*qubits))
circuit

0: ───H───@───M───
          │   │
1: ───────X───M───

In [12]:
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│───S^-1───│───DD────────S──────│─────────────Z────────────────│───DD────────Z──────│───M('q(0),q(1)')───
           │          │   │                │                              │   │                │   │
0-: ───X───│──────────│───DD^-0.5──────────│───DD────────S──────DD────────│───DD───────────────│───M────────────────
           │          │                    │   │                │         │                    │   │
1+: ───────│───S──────│───DD────────S^-1───│───┼─────────S^-1───┼─────────│───DD────────S^-1───│───M────────────────
           │          │   │                │   │                │         │   │                │   │
1-: ───X───│──────────│───DD^-0.5──────────│───DD^0.25───S^-1───DD^0.25───│───DD^-0.5──────────│───M────────────────

In [13]:
gate_domain = {
    cirq.X: 1,
    cirq.Y: 1,
    cirq.Z: 1,
    cirq.S: 1,
    cirq.T: 1,
    cirq.H: 1,
    cirq.rx(1.23): 1,
    cirq.ry(2.34): 1,
    cirq.CZ: 2,
    cirq.CX: 2,
    cirq.CX**0.5: 2,
    cirq.SWAP: 2,
    cirq.ISWAP: 2,
    css.ZZSwapGate(1.23): 2,
    css.Barrier(3): 3,
}

In [14]:
n, depth, op_density = (4, 8, 0.8)
qubits = cirq.LineQubit.range(n)
circuit = cirq.testing.random_circuit(qubits, depth, op_density, gate_domain=gate_domain)
circuit

0: ───ZZSwap(0.392π)─────────────────────────────────────────────@───│───
      │                                                          │   │
1: ───ZZSwap(0.392π)───@───────X^0.5───Ry(0.745π)────────────────@───┼───
                       │       │                                     │
2: ────────────────────X───────┼────────────────────Ry(0.745π)───────│───
                               │                                     │
3: ────────────────────────────@────────────────────X────────────────│───

In [15]:
circuit.insert(depth // 2, css.barrier(*qubits))
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│───Z^-0.75───│───DD────────Z^0.75───│─────────────Z^0.75─────────────│───DD────────Z^-0.75───│─────────────Z^0.75─────────────│───DD──────────T─────────│─────────────T^-1────────────────│───DD────────T──────│──────────────────────────────│───────────────────│──────────────────────────────│──────────────────│─────────────────────────────────│───────────────────│──────────│───────────────────│─────────────Z────────────────│───
           │             │   │                  │                                │   │                   │                                │   │                     │                                 │   │                │                              │                   │                              │                  │                                 │                   │          │                   │                              │
0-: ───X───│─────────────│───DD^-0.5────────────│───DD────────S────────DD────────│───DD^-0.5─────────────│───DD────────S^-1─────DD────────│───DD^-0.5───────────────│───DD────────S─────────DD────────│───DD───────────────│──────────────────────────────│───────────────────│──────────────────────────────│──────────────────│─────────────────────────────────│───────────────────│──────────│───────────────────│───DD────────S^-1───DD────────│───
           │             │                      │   │                  │         │                       │   │                  │         │                         │   │                   │         │                    │                              │                   │                              │                  │                                 │                   │          │                   │   │                │         │
1+: ───────│───T^-1──────│───DD────────T────────│───┼─────────Z^0.75───┼─────────│───DD────────T─────────│───┼─────────Z^0.75───┼─────────│───DD──────────Z^-0.75───│───┼─────────Z^-0.75───┼─────────│───DD────────T^-1───│─────────────S^-1─────────────│───DD──────────S───│─────────────S^-1─────────────│───DD─────────S───│─────────────S^-1────────────────│───DD──────────Z───│───S^-1───│───DD──────────S───│───┼────────────────┼─────────┼───
           │             │   │                  │   │                  │         │   │                   │   │                  │         │   │                     │   │                   │         │   │                │                              │   │               │                              │   │              │                                 │   │               │          │   │               │   │                │         │
1-: ───X───│─────────────│───DD^-0.5────────────│───DD^0.25───S^-1─────DD^0.25───│───DD^-0.5─────────────│───DD^0.25───S────────DD^0.25───│───DD^-0.892─────────────│───DD^0.25───S^-1──────DD^0.25───│───DD^-0.5──────────│───DD────────S──────DD────────│───DD^-0.906───────│───DD────────S──────DD────────│───DD^-0.25───────│───DD────────S─────────DD────────│───DD^-0.844───────│──────────│───DD^-0.745───────│───DD^0.25───S──────DD^0.25───┼───
           │             │                      │                                │                       │                                │                         │                                 │                    │   │                │         │                   │   │                │         │                  │   │                   │         │                   │          │                   │                              │
2+: ───────│───S─────────│───DD────────S^-1─────│────────────────────────────────│───────────────────────│────────────────────────────────│─────────────────────────│─────────────────────────────────│────────────────────│───┼─────────S──────┼─────────│───DD──────────S───│───┼────────────────┼─────────│──────────────────│───┼───────────────────┼─────────│───────────────────│───S^-1───│───DD──────────S───│──────────────────────────────│───
           │             │   │             

In [16]:
mat = compute_unitary(compiled_circuit)
cirq.testing.assert_allclose_up_to_global_phase(cirq.unitary(circuit), mat, atol=1e-8)

# Circuit Simulation

In [23]:
n, depth, op_density = (2, 3, 0.8)
qubits = cirq.LineQubit.range(n)
circuit = cirq.testing.random_circuit(qubits, depth, op_density, gate_domain=gate_domain)
circuit+= cirq.measure(*qubits)
circuit

0: ───Y───────@───M───
              │   │
1: ───────X───@───M───

In [24]:
service = css.Service()
job = service.create_job(circuit, target = "eeroq_wonderlake_qpu", repetitions=100, method="noise-sim", error_rate=0.01)

In [25]:
job.counts()

[{'11': 97, '22': 1, '01': 1, '21': 1}]

A measurement value of 2 refers to an out-of-codespace error.